In [18]:
import os
import cv2
import nrrd
import numpy as np
import SimpleITK as sitk

In [36]:
def create_retouch_files(path,end_folder):
    final_folder=end_folder+'/'+'RETOCUH_TRAINING'
    os.mkdir(final_folder)
    
    train_folder_path=final_folder+'/'+'imagesTr'
    label_folder_path=final_folder+'/'+'labelsTr'
    os.mkdir(train_folder_path)
    os.mkdir(label_folder_path)
    
    for folder in os.listdir(path):
        training_paths=path+'/'+folder
        for in_folder in os.listdir(training_paths):
            train_paths=training_paths+'/'+in_folder
            patient_number=int(in_folder[5:])
            for file in os.listdir(train_paths):
                if file == 'oct.mhd':
                    file_path=train_paths+'/'+file
                    in_folders(train_folder_path,file_path,patient_number)
                if file == 'reference.mhd':
                    file_path=train_paths+'/'+file
                    in_folders(label_folder_path,file_path,patient_number)

In [30]:
# NRRD fájl beolvasása
def get_data(fajl_utvonal):
    try:
        data, header = nrrd.read(fajl_utvonal)
    except Exception as e:
        print("Hiba történt a fájl beolvasása közben:")
        print(e)
    return data

In [33]:
def mask_image_by_values(image, allowed_values):
    allowed_values = np.array(allowed_values)
    mask = np.isin(image, allowed_values)
    masked_image = np.where(mask, image, 0)

    # Értékek cseréje
    masked_image[masked_image == 5] = 3
    masked_image[masked_image == 6] = 2
    masked_image[masked_image == 7] = 1

    return masked_image

In [34]:
def create_aroi_npy(folderpath,validate_path,isitlabel):
    allowed=[0,5,6,7]
    files = os.listdir(folderpath)
    files.sort()
    for i,file in enumerate(files):
        j=i+1
        utvonal= folderpath + '/' + file
        print(j,utvonal)
        data = get_data(utvonal)
        new_folder = validate_path+'/'+str(j)
        os.mkdir(new_folder)
        data = get_data(utvonal)
        slice_dimension= data.shape[2]
        for slice in range(slice_dimension):
            szelet = data[:,:,slice]
            #label esetén más
            szelet = cv2.rotate(szelet, cv2.ROTATE_90_CLOCKWISE)
            if isitlabel:
                szelet = mask_image_by_values(szelet,allowed)
            save_path=new_folder+'/'+str(j)+'_'+str(slice)+'.npy'
            with open(save_path, 'wb') as f:
                np.save(f,szelet)

In [22]:
def create_aroi_files(data_path,destination_path):
    aroi_training=destination_path+'/'+'AROI_TRAINING'
    imagestr=aroi_training+'/'+'imagesTr'
    labelstr=aroi_training+'/'+'labelsTr'

    os.mkdir(aroi_training)
    os.mkdir(imagestr)
    os.mkdir(labelstr)

    data_imagestr=data_path+'/'+'imagesTr'
    data_labelstr=data_path+'/'+'labelsTr'
    create_aroi_npy(data_imagestr,imagestr,False)
    create_aroi_npy(data_labelstr,labelstr,True)

In [ ]:
#Az alábbi két eljárás létrehozza az előre megadott mappákban npy-formátumban, patient_number-ként szétosztva a fájlokat
#A RETOUCH adatoknak az alábbi mappaszerkezetben kell lenniük
#
#Mappanév pl:training_sets
#
# training_sets:
#   RETOUCH-TrainingSetCirrus
#   RETOUCH-TrainingSet-Spectralis
#   RETOUCH-TrainingSet-Topcon

#Az AROI adatoknak az alábbi mappaszerkezetben kell elhelyezkediük
#
# AROI_DATA:
#   imagesTr
#   labelsTr

#A destionation_folder egy tetszőlegesen kiválasztott meghajtó vagy 

AROI_FOLDER=''
RETOUCH_FOLDER=''

create_aroi_files(AROI_FOLDER,destination_path)
create_retouch_files(RETOUCH_FOLDER,destination_folder)